In [1188]:
print("Best accuracy for now on cross val (10-split) : 0.8290347293156282 %")

Best accuracy for now on cross val (10-split) : 0.8290347293156282 %


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

## Read the data

In [7]:
data = pd.read_csv('data/train.csv')
data = data.sample(frac=1).reset_index(drop=True)

In [1191]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,581,1,2,"Christy, Miss. Julie Rachel",female,25.0,1,1,237789,30.0000,NaN,S
1,378,0,1,"Widener, Mr. Harry Elkins",male,27.0,0,2,113503,211.5000,C82,C
2,651,0,3,"Mitkoff, Mr. Mito",male,NaN,0,0,349221,7.8958,NaN,S
3,424,0,3,"Danbom, Mrs. Ernst Gilbert (Anna Sigrid Maria ...",female,28.0,1,1,347080,14.4000,NaN,S
4,294,0,3,"Haas, Miss. Aloisia",female,24.0,0,0,349236,8.8500,NaN,S


## Exploring the data and the features

In [1192]:
print("Pourcentage of survived :")
print(sum(data.Survived==1)/len(data),'%')

Pourcentage of survived :
0.3838383838383838 %


In [1193]:
print("Pourcentage of female survived :")
print(len(data[data["Sex"]=='female'][data['Survived']==1])/len(data.dropna(subset=['Sex'])[data['Survived']==1]),'%')

Pourcentage of female survived :
0.6812865497076024 %


/Users/antoine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [1194]:
print("Pourcentage of chield under 16 survived:")
print(len(data[data["Age"]<16][data['Survived']==1])/len(data.dropna(subset=['Age'])[data['Survived']==1]),'%')

Pourcentage of chield under 16 survived:


/Users/antoine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


0.16896551724137931 %


In [1195]:
print("Pourcentage of 1st class survived:")
print(len(data[data["Pclass"]==1][data['Survived']==1])/len(data.dropna(subset=['Pclass'])[data['Survived']==1]),'%')

Pourcentage of 1st class survived:
0.39766081871345027 %


/Users/antoine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [1196]:
print("Pourcentage of 2nd class survived:")
print(len(data[data["Pclass"]==2][data['Survived']==1])/len(data.dropna(subset=['Pclass'])[data['Survived']==1]),'%')

Pourcentage of 2nd class survived:
0.2543859649122807 %


/Users/antoine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [1197]:
prbint("Pourcentage of 3rd class survived:")
print(len(data[data["Pclass"]==3][data['Survived']==1])/len(data.dropna(subset=['Pclass'])[data['Survived']==1]),'%')

Pourcentage of 3rd class survived:
0.347953216374269 %


/Users/antoine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [8]:
print("Best chance to survive:")
print(len(data[data["Pclass"]==1][data['Survived']==1][data['Sex']=='female'][data['Age']>16])/len(data[data["Pclass"]==1][data['Sex']=='female'][data['Age']>16]),'%')

Pourcentage of 3rd class survived:
0.9746835443037974 %


/Users/antoine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


## Prepare the data train

In [1324]:
data = pd.read_csv('titanic/train.csv')

print(data.shape)
print(data.columns)

(891, 12)
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


Feel the NaN of the age with random gaussian

In [1325]:
mean_age = data.Age.mean()
std_age = data.Age.std()
data.Age = data.Age.mask(data.Age.isnull(),np.random.normal(mean_age,std_age, size=data.Age.shape))

One hot encoding for Sex = female/male

In [1326]:
one_hot = pd.get_dummies(data['Sex'],prefix='sex')
data = data.drop('Sex',axis = 1)
data = data.join(one_hot)

One hot encoding for Embarked = C/Q/S

In [1327]:
one_hot = pd.get_dummies(data['Embarked'],prefix='embarked')
data = data.drop('Embarked',axis = 1)
data = data.join(one_hot)

Add features Children or Adult

In [1328]:
children_column = pd.Series((data.Age < 16).astype(int),name="Children")
data = data.join(children_column)

Add feature if travel alone or not

In [1329]:
family_column = pd.Series(((data['SibSp'] + data['Parch'])>=1).astype(int),name='Family')
data = data.join(family_column)

We do not need to normalize the data because we use tree-based algorithms

In [1330]:
#data.Pclass = (data.Pclass-1)/2
#data.Age = data.Age-mean_age)/std_age

In [1331]:
data.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,Children,Family
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0,1,0,0,1,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1,0,1,0,0,0,1
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,0,0,0,1,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,0,0,0,1,0,1
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0,1,0,0,1,0,0


In [1332]:
#features=['Pclass','Age','Fare','SibSp','Parch','sex_male','embarked_C','embarked_Q','embarked_S','Children','Family']
#features = ['Pclass','Age','Fare','sex_male','embarked_C','embarked_Q','embarked_S','Children','Family']
features = ['Pclass','Fare','sex_male','embarked_C','embarked_Q','embarked_S','Children','Family']

In [1333]:
X = data[features]
X.head()

,Pclass,Fare,sex_male,embarked_C,embarked_Q,embarked_S,Children,Family
0,3,7.2500,1,0,0,1,0,1
1,1,71.2833,0,1,0,0,0,1
2,3,7.9250,0,0,0,1,0,0
3,1,53.1000,0,0,0,1,0,1
4,3,8.0500,1,0,0,1,0,0


In [1334]:
y = data.Survived

## Linear Regression

In [918]:
from sklearn.model_selection import train_test_split

### SVM

In [919]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import zero_one_loss

In [920]:
linear_regression_model1 = SGDClassifier()

In [922]:
linear_regression_model1.fit(X_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [923]:
y_res = linear_regression_model1.predict(X_test)

In [924]:
rate = zero_one_loss(y_res,y_test)
print("Accuracy : ",1-rate,'%')

Accuracy :  0.6971830985915493 %


### Ridge Classifier

In [925]:
from sklearn.linear_model import RidgeClassifier

In [926]:
ridge_model1 = RidgeClassifier()

In [927]:
ridge_model1.fit(X_train,y_train)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [928]:
y_res = ridge_model1.predict(X_test)

In [929]:
rate = zero_one_loss(y_res,y_test)
print("Accuracy : ",1-rate,'%')

Accuracy :  0.7887323943661971 %


## Logistic Regression

In [1091]:
from sklearn.linear_model import LogisticRegression

In [1094]:
n_three = [10,50,100,200]
depth = [2,5,10,15,20]
n_max = 1
d_max = 1
best_score = 0.0
index = np.arange(len(X))
kf = KFold(n_splits=10)
mean_score = 0.0
for train, test in kf.split(index):
    X_train, X_test, y_train, y_test = X.iloc[train,:], X.iloc[test,:], y.iloc[train], y.iloc[test]
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train,y_train)
    y_res = logistic_model.predict(X_test)
    rate = zero_one_loss(y_res,y_test)
    mean_score += (1-rate)
mean_score = mean_score/10
print("Accuracy : ",mean_score,'%')

Accuracy :  0.7941777323799796 %


/Users/antoine/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/antoine/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/antoine/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/antoine/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/antoine/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed

In [933]:
y_res = logistic_model1.predict(X_test)

In [976]:
rate = zero_one_loss(y_res,y_test)
print("Accuracy : ",1-rate,'%')

Accuracy :  0.8169014084507042 %


In [935]:
logistic_model1.fit(X,y)

/Users/antoine/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [936]:
predictions = logistic_model1.predict(X_res)

In [937]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


## Random Forest

In [1321]:
from sklearn.ensemble import RandomForestClassifier

In [1335]:
n_three = [10,50,100,200]
depth = [2,5,10,15,20]
n_max = 1
d_max = 1
best_score = 0.0
for n in n_three:
    for d in depth:
        len(data)
        index = np.arange(len(X))
        kf = KFold(n_splits=10)
        mean_score = 0.0
        for train, test in kf.split(index):
            X_train, X_test, y_train, y_test = X.iloc[train,:], X.iloc[test,:], y.iloc[train], y.iloc[test]
            random_forest_model = RandomForestClassifier(n_estimators=n, max_depth=d)
            random_forest_model.fit(X_train, y_train)
            y_res = random_forest_model.predict(X_test)
            rate = zero_one_loss(y_res,y_test)
            mean_score += (1-rate)
        mean_score = mean_score/10
        print("For",n,'three of depth',d,':')
        print("Accuracy : ",mean_score,'%')
        if (mean_score > best_score):
            n_max = n
            d_max = d
            best_score = mean_score
print("Best accurate is given for ",n_max,"three of depth",d_max,"with an accurency of",best_score,"% in cross validation")

For 10 three of depth 2 :
Accuracy :  0.7655305867665418 %
For 10 three of depth 5 :
Accuracy :  0.8002372034956305 %
For 10 three of depth 10 :
Accuracy :  0.809250936329588 %
For 10 three of depth 15 :
Accuracy :  0.8126092384519351 %
For 10 three of depth 20 :
Accuracy :  0.810362047440699 %
For 50 three of depth 2 :
Accuracy :  0.7801123595505619 %
For 50 three of depth 5 :
Accuracy :  0.8170536828963796 %
For 50 three of depth 10 :
Accuracy :  0.8002372034956304 %
For 50 three of depth 15 :
Accuracy :  0.8070037453183521 %
For 50 three of depth 20 :
Accuracy :  0.8036329588014981 %
For 100 three of depth 2 :
Accuracy :  0.7700124843945069 %
For 100 three of depth 5 :
Accuracy :  0.8193133583021224 %
For 100 three of depth 10 :
Accuracy :  0.8148564294631712 %
For 100 three of depth 15 :
Accuracy :  0.8036329588014981 %
For 100 three of depth 20 :
Accuracy :  0.8070037453183521 %
For 200 three of depth 2 :
Accuracy :  0.7812359550561798 %
For 200 three of depth 5 :
Accuracy :  0.81

In [1323]:
random_forest_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1)
random_forest_model.fit(X, y)
y_res = random_forest_model.predict(X_test)
rate = zero_one_loss(y_res,y_test)
print("For",n,'three of depth',d,':')
print("Accuracy : ",1-rate,'%')

For 200 three of depth 20 :
Accuracy :  0.9662921348314607 %


## Prepare the data for the test

In [1275]:
data_test = pd.read_csv('data/test.csv')

In [1276]:
data_test.Age = data_test.Age.mask(data_test.Age.isnull(),np.random.normal(data.Age.mean(),data.Age.std(), size=data_test.Age.shape))

In [1277]:
one_hot = pd.get_dummies(data_test['Sex'],prefix='sex')
data_test = data_test.drop('Sex',axis = 1)
data_test = data_test.join(one_hot)

In [1278]:
one_hot = pd.get_dummies(data_test['Embarked'],prefix='embarked')
data_test = data_test.drop('Embarked',axis = 1)
data_test = data_test.join(one_hot)

In [1279]:
children_column = pd.Series((data_test.Age < 16).astype(int),name="Children")
data_test = data_test.join(children_column)

In [1280]:
family_column = pd.Series(((data_test['SibSp'] + data_test['Parch'])>=1).astype(int),name='Family')
data_test = data_test.join(family_column)

In [1295]:
data_test.Fare = data_test.Fare.fillna(data_test.Fare.mean())

In [1296]:
X_test = data_test[features]

In [1297]:
(X_test.Fare.fillna(3) == 'NaN').sum()

/Users/antoine/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


0

In [1298]:
(X_test.Fare == 'Nan').sum() 

0

In [1299]:
X_test.head()

,Pclass,Age,Fare,SibSp,Parch,sex_male,embarked_C,embarked_Q,embarked_S,Children,Family
0,3,34.5,7.8292,0,0,1,0,1,0,0,0
1,3,47.0,7.0000,1,0,0,0,0,1,0,1
2,2,62.0,9.6875,0,0,1,0,1,0,0,0
3,3,27.0,8.6625,0,0,1,0,0,1,0,0
4,3,22.0,12.2875,1,1,0,0,0,1,0,1


In [1301]:
X_test[X_test.isna().any(axis=1)]

,Pclass,Age,Fare,SibSp,Parch,sex_male,embarked_C,embarked_Q,embarked_S,Children,Family


# Submissions

In [1305]:
random_forest_model = RandomForestClassifier(n_estimators=50, max_depth=5)
random_forest_model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [1338]:
predictions = random_forest_model.predict(X_test)

In [1337]:
test_data = pd.read_csv('titanic/test.csv')
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


<class 'sklearn.model_selection._split.KFold'>
